In [ ]:
# first of all, let's check that avro tools work

!java -jar /usr/bin/avro.jar

In [3]:
# check that Spark client works

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.sql.warehouse.dir", "/usr/local/hadoop/warehouse") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hivemetastore:9083") \
    .config("spark.sql.catalog.iceberg.cache-enabled", False) \
    .getOrCreate()

spark.sql("""
    SHOW TABLES FROM iceberg.default
""").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



25/02/09 10:17:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# check that Trino client works

from trino.dbapi import connect

trino_connection = connect(
    host="trino",
    port=8080,
    user="iceberg",
)
trino = trino_connection.cursor()
trino.execute("SELECT * FROM system.runtime.nodes")

rows = trino.fetchall()
print(rows)

[['fab022e1f9a2', 'http://172.18.0.2:8080', '457', True, 'active']]


In [5]:
# check that Python client works

from pyiceberg.catalog.hive import HiveCatalog
from pyiceberg.exceptions import NoSuchTableError

catalog = HiveCatalog(name="iceberg", uri="thrift://hivemetastore:9083")

try:
    catalog.load_table("default.sample")
except NoSuchTableError as e:
    print(e)

Table does not exists: sample
